In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import os
from pathlib import Path
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


# IGNORING UNNECESSARRY WARNINGS

filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)

In [4]:
No_Data_Path = Path("/kaggle/input/brain-mri-images-for-brain-tumor-detection/no")
Yes_Data_Path = Path("/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes")


In [5]:
No_JPG_Path = list(No_Data_Path.glob(r"*.jpg"))
Yes_JPG_Path = list(Yes_Data_Path.glob(r"*.jpg"))

In [14]:
JPG_Path_List = []
JPG_Labels = []

for No_JPG in No_JPG_Path:
    JPG_Path_List.append(str(No_JPG))
    JPG_Labels.append('No') 
    
for Yes_JPG in Yes_JPG_Path:
    JPG_Path_List.append(str(Yes_JPG))
    JPG_Labels.append('Yes') 

In [15]:
Main_Train_Data = pd.DataFrame({
    'JPG': JPG_Path_List,
    'TUMOR_CATEGORY': JPG_Labels
})


In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    Main_Train_Data['JPG'],
    Main_Train_Data['TUMOR_CATEGORY'],
    test_size=0.2,
    random_state=42)

In [17]:
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))  
    img = img / 255.0  
    return img

In [18]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=Main_Train_Data,
    x_col='JPG',
    y_col='TUMOR_CATEGORY',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training',
    seed=42)

Found 171 validated image filenames belonging to 2 classes.


In [19]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [22]:
train_data, validation_data = train_test_split(
    Main_Train_Data,
    test_size=0.2,
    random_state=42
)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    x_col='JPG',
    y_col='TUMOR_CATEGORY',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    seed=42
)


validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_dataframe(
    dataframe=validation_data,
    x_col='JPG',
    y_col='TUMOR_CATEGORY',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False  
)

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Found 136 validated image filenames belonging to 2 classes.
Found 35 validated image filenames belonging to 2 classes.
Epoch 1/10
5/5 [==============================] - 11s 2s/step - loss: 3.1614 - accuracy: 0.5000 - val_loss: 1.0236 - val_accuracy: 0.5143
Epoch 2/10
5/5 [==============================] - 9s 2s/step - loss: 0.6594 - accuracy: 0.6618 - val_loss: 0.6215 - val_accuracy: 0.6000
Epoch 3/10
5/5 [==============================] - 8s 2s/step - loss: 0.6398 - accuracy: 0.6176 - val_loss: 0.5399 - val_accuracy: 0.6857
Epoch 4/10
5/5 [==============================] - 8s 1s/step - loss: 0.6079 - accuracy: 0.6838 - val_loss: 0.5258 - val_accuracy: 0.7429
Epoch 5/10
5/5 [==============================] - 8s 2s/step - loss: 0.6309 - accuracy: 0.6618 - val_loss: 0.5417 - val_accuracy: 0.7143
Epoch 6/10
5/5 [==============================] - 8s 1s/step - loss: 0.5798 - accuracy: 0.6912 - val_loss: 0.5874 - val_accuracy: 0.7143
Epoch 7/10
5/5 [==============================] - 8s 1s/st

In [27]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=Main_Train_Data,
    x_col='JPG',
    y_col='TUMOR_CATEGORY',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False  
)

test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy:", test_acc)

Found 171 validated image filenames belonging to 2 classes.
6/6 [==============================] - 2s 343ms/step - loss: 0.3986 - accuracy: 0.8421
Test Accuracy: 0.8421052694320679
